# Парсинг сайта banki.ru

### Установка библиотек
Если вы не знаете установлены эти библиотеки или нет, то установите еще раз (второй раз установки ничего не сломает)

In [1]:
# pip install requests
# pip install pandas
# pip install time
# pip install ssl
# pip install bs4
# pip install openpyxl

### Импорты библиотек

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import ssl
import time as t

### Код с опробированием методов

In [3]:
url = 'https://banki.ru/insurance/responses/company/sberbankstrahovaniezhizni'
r = requests.get(url)

In [4]:
r

<Response [200]>

In [25]:
r.text[:1000]

'<!doctype html>\n<html class="env-no-js env-desktop "\n\tprefix="ya: http://webmaster.yandex.ru/vocabularies/\nog: http://ogp.me/ns#" xmlns="http://www.w3.org/1999/xhtml" xmlns:og="http://opengraphprotocol.org/schema/" xmlns:fb="http://www.facebook.com/2008/fbml">\n\t<head >\n\t\t\t\t\t<link rel="shortcut icon" href="/static/favicons/favicon.ico">\n<link rel="icon" type="image/png" sizes="16x16" href="/static/favicons/favicon-16x16.png">\n<link rel="icon" type="image/png" sizes="32x32" href="/static/favicons/favicon-32x32.png">\n\n<meta name="mobile-web-app-capable" content="yes">\n<meta name="theme-color" content="#fff">\n<meta name="application-name" content="Banki.ru — финансовый супермаркет">\n<link rel="apple-touch-icon" sizes="57x57" href="/static/favicons/apple-touch-icon-57x57.png">\n<link rel="apple-touch-icon" sizes="60x60" href="/static/favicons/apple-touch-icon-60x60.png">\n<link rel="apple-touch-icon" sizes="72x72" href="/static/favicons/apple-touch-icon-72x72.png">\n<lin

In [27]:
soup = BeautifulSoup(r.text)
article = soup.find('article')
article

<article class="responses__item" data-id="3246" data-state="collapsed" data-test="responses-item" itemscope="" itemtype="http://data-vocabulary.org/Review">
<meta content="Сбербанк страхование" itemprop="itemreviewed"/>
<div class="flexbox flexbox--row flexbox--gap_xxsmall flexbox--align-items_flex-start">
<a class="header-h3" data-test="responses-header" href="/insurance/responses/company/response/3246/" itemprop="summary">Смещение сроков составления страхового акта, без предупреждения</a>
</div>
<div class="margin-top-xx-small color-gray-burn">
<div class="flexbox flexbox--inline flexbox--row flexbox--gap_xsmall flexbox--align-items_baseline" itemprop="reviewRating" itemscope="" itemtype="http://schema.org/Rating">
<span class="text-label" data-test="responses-estimate">Оценка:</span>
<span class="rating-grade rating-grade--small rating-grade--value-1" data-test="responses-rating-grade" itemprop="rating">
			1
		</span>
<span class="text-label" data-test="responses-status">
</span>
<

In [14]:
header = article.find('a', {'data-test': 'responses-header'}).text
header

'страхование жизни'

In [15]:
href = article.find('a', {'data-test': 'responses-header'}).get('href')
href

'/insurance/responses/company/response/28235/'

In [17]:
href = article.find('a', {'data-test': 'responses-header'}).get('href')
header = article.find('a', {'data-test': 'responses-header'}).text
text = article.find('div', {'itemprop': 'description'}).text.strip()
time = article.find('time', {'data-test': 'responses-datetime', 'itemprop': 'dtreviewed'}).get('datetime')

try:
    rating = article.find('span', {'data-test': 'responses-rating-grade'}).text.strip()
except:
    rating = None
try:
    rating_payouts = article.find('strong', {'class': 'font-size-medium'}).text.strip()
except:
    rating_payouts = None
try:
    rating_status = article.find('span', {'data-test': 'responses-status'}).text.strip()
except:
    rating_status = None

href, header, text, time, rating, rating_payouts, rating_status

('/insurance/responses/company/response/28235/',
 'страхование жизни',
 'ООО СК "Сбербанк"-организация-лохотрон!!!! Люди, опомнитесь, какое страхование,\xa0 заключив такого рода договор\xa0 вы содержите государство! в лице страховой компании Сбербанк! Никаких процентов, премий страховых выплачено\xa0 НЕ БУДЕТ!!!!! Это обман!Пенсионеры! у вас слишком много денег, не знаете как их потратить!? Живите в свое удовольствие! ни в коем случае не заключайте договоров со Сбербанком!',
 '2020-04-21 21:12:47',
 None,
 None,
 None)

In [28]:
df = pd.DataFrame(columns=[
    'url жалобы', 'Заголовок', 'Статус', 'Текст', 'Время', 'Оценка', 'Оценка выплат',
])

for article in soup.findAll('article'):
    try:
        href = article.find('a', {'data-test': 'responses-header'}).get('href')
        header = article.find('a', {'data-test': 'responses-header'}).text
        text = article.find('div', {'itemprop': 'description'}).text.strip()
        time = article.find('time', {'data-test': 'responses-datetime', 'itemprop': 'dtreviewed'}).get('datetime')

        # optional
        try:
            rating = article.find('span', {'data-test': 'responses-rating-grade'}).text.strip()
        except:
            rating = None
        try:
            rating_payouts = article.find('strong', {'class': 'font-size-medium'}).text.strip()
        except:
            rating_payouts = None
        try:
            rating_status = article.find('span', {'data-test': 'responses-status'}).text.strip()
        except:
            rating_status = None

        row = [href, header, rating_status, text, time, rating, rating_payouts]

        df.loc[len(df)] = row
    except:
        pass
    
df.head()

,url жалобы,Заголовок,Статус,Текст,Время,Оценка,Оценка выплат
0,/insurance/responses/company/response/3246/,"Смещение сроков составления страхового акта, б...",,День добрый.13.01.17г. у меня произошел страхо...,2017-02-20 09:39:51,1,None
1,/insurance/responses/company/response/3160/,Страхование без выплат,Оценка по выплатам:\n1,"Компанию ООО ""СБЕРБАНК СТРАХОВАНИЕ"" выбрал, т...",2017-02-08 13:51:55,None,1
2,/insurance/responses/company/response/3146/,"Игнорирование сотрудниками страховой, некомпет...",,"Добрый день.27 октября 2016 года, обратившись ...",2017-02-06 17:35:09,1,None
3,/insurance/responses/company/response/3116/,Не можем получить страховую выплату от СБЕРБАН...,,30 января 2017 года у меня Хатламаджиева А. М....,2017-02-02 11:56:43,1,None
4,/insurance/responses/company/response/3110/,Не возвращают страховую премию при расторжении...,Проблема решена,"Здравствуйте, но сложилась вопиющая ситуация, ...",2017-02-01 15:44:41,2,None


### Объединенный код 

In [23]:
ssl._create_default_https_context = ssl._create_unverified_context # чтобы

base_url = 'https://banki.ru/'

companies = {
    'СБСЖ': {
        'pages': 50,  
        'url': 'insurance/responses/company/sberbankstrahovaniezhizni/',
    }, # СБСЖ 
    'СБС': {
        'pages': 16,
        'url': 'insurance/responses/company/sberbankstrahovanie/',
    }, # СБС 
}

page_url = '?page='

df = pd.DataFrame(columns=[
    'Компания', 'url жалобы', 'Заголовок', 'Статус', 'Текст', 'Время', 'Оценка', 'Оценка выплат',
])

for company in companies.keys():
    print(company)

    company_url = companies[company]['url']
    r = requests.get(base_url + company_url)
    soup = BeautifulSoup(r.text)

    pages = companies[company]['pages']

    for page in range(2, pages + 2):
        t.sleep(2)

        for article in soup.findAll('article'):
            try:
                href = article.find('a', {'data-test': 'responses-header'}).get('href')
                header = article.find('a', {'data-test': 'responses-header'}).text
                text = article.find('div', {'itemprop': 'description'}).text.strip()
                time = article.find('time', {'data-test': 'responses-datetime', 'itemprop': 'dtreviewed'}).get('datetime')
                
                # optional
                try:
                    rating = article.find('span', {'data-test': 'responses-rating-grade'}).text.strip()
                except:
                    rating = None
                try:
                    rating_payouts = article.find('strong', {'class': 'font-size-medium'}).text.strip()
                except:
                    rating_payouts = None
                try:
                    rating_status = article.find('span', {'data-test': 'responses-status'}).text.strip()
                except:
                    rating_status = None

                row = [company, base_url+href[1:], header, rating_status, text, time, rating, rating_payouts]

                df.loc[len(df)] = row
            except:
                pass

        # Взятие следующей страницы
        try:
            if page <= pages:
                r = requests.get(base_url + company_url + page_url + str(page))
                soup = BeautifulSoup(r.text, features='lxml')
        except:
            pass

print(len(df))
df.to_excel('Отзывы_страхование.xlsx')

СБСЖ
СБС
1634
